# RePath Model Retraining Workflow\n\nThis notebook walks through benchmark prep, annotation seeding, candidate training/export, and candidate evaluation.

## 1. Paths and Environment

In [ ]:
import os\nimport subprocess\nfrom pathlib import Path\n\nROOT = Path.cwd()\nMOBILE = (ROOT / "../repath-mobile").resolve()\nKAGGLE_DIR = os.environ.get("KAGGLE_WASTE_DIR", "")\n\nprint({\n    "repath_model": str(ROOT),\n    "repath_mobile": str(MOBILE),\n    "kaggle_dir": KAGGLE_DIR,\n})

## 2. Benchmark Prep and Coverage\nRun sync, holdout build, resolved manifest build, coverage, and audit checks.

In [ ]:
subprocess.run([\n    "python3", "scripts/sync_benchmark_progress.py",\n    "--manifest", str(MOBILE / "test/benchmarks/municipal-benchmark-manifest-v2.json"),\n    "--completed", str(MOBILE / "test/benchmarks/benchmark-labeled.csv"),\n], check=True)

In [ ]:
cmd = [\n    "python3", "scripts/build_supported_holdout_manifest.py",\n    "--labels", str(MOBILE / "assets/models/yolo-repath.labels.json"),\n    "--input-csv", str(MOBILE / "test/benchmarks/benchmark-labeled.csv"),\n    "--retraining-manifest", str(MOBILE / "ml/artifacts/retraining/retraining-manifest.json"),\n    "--cache-dir", str(MOBILE / "test/benchmarks/images/supported-holdout"),\n    "--out", str(MOBILE / "test/benchmarks/benchmark-manifest.supported-holdout.json"),\n]\nif KAGGLE_DIR:\n    cmd.extend(["--kaggle-dir", KAGGLE_DIR])\nsubprocess.run(cmd, check=True)

In [ ]:
subprocess.run([\n    "python3", "scripts/build_resolved_benchmark_manifest.py",\n    "--manifest", str(MOBILE / "test/benchmarks/municipal-benchmark-manifest-v2.json"),\n    "--completed", str(MOBILE / "test/benchmarks/benchmark-labeled.csv"),\n    "--cache-dir", str(MOBILE / "test/benchmarks/images"),\n    "--out", str(MOBILE / "test/benchmarks/municipal-benchmark-manifest.resolved.json"),\n], check=True)

In [ ]:
subprocess.run([\n    "python3", "scripts/check_benchmark_coverage.py",\n    "--taxonomy", str(MOBILE / "assets/models/municipal-taxonomy-v1.json"),\n    "--manifest", str(MOBILE / "test/benchmarks/municipal-benchmark-manifest.resolved.json"),\n    "--out", str(MOBILE / "test/benchmarks/benchmark-coverage-report.resolved.json"),\n], check=True)\n\nsubprocess.run([\n    "python3", "scripts/audit_benchmark_dataset.py",\n    "--manifest", str(MOBILE / "test/benchmarks/municipal-benchmark-manifest.resolved.json"),\n    "--taxonomy", str(MOBILE / "assets/models/municipal-taxonomy-v1.json"),\n    "--out", str(MOBILE / "test/benchmarks/benchmark-dataset-audit.resolved.json"),\n], check=True)

## 3. Annotation + Training\nThese commands expect annotation bundle content in `repath-mobile/ml/artifacts/retraining/annotation-bundle`.

In [ ]:
subprocess.run([\n    "python3", "scripts/seed_annotation_boxes.py",\n    "--bundle-root", str(MOBILE / "ml/artifacts/retraining/annotation-bundle"),\n    "--model", str(MOBILE / "assets/models/yolo-repath.tflite"),\n    "--labels", str(MOBILE / "assets/models/yolo-repath.labels.json"),\n], check=True)

In [ ]:
# Optional quick dry run\nsubprocess.run([\n    "python3", "scripts/train_detector_from_annotation.py",\n    "--bundle-root", str(MOBILE / "ml/artifacts/retraining/annotation-bundle"),\n    "--candidate-root", str(MOBILE / "ml/artifacts/models/candidates"),\n    "--project", str(MOBILE / "ml/artifacts/training-runs"),\n    "--dry-run",\n], check=True)

## 4. Candidate Evaluation\nRun candidate benchmark + analysis + comparison against baseline.

In [ ]:
subprocess.run([\n    "python3", "scripts/benchmark_candidate_model.py",\n    "--candidates-root", str(MOBILE / "ml/artifacts/models/candidates"),\n    "--manifest", str(MOBILE / "test/benchmarks/municipal-benchmark-manifest.resolved.json"),\n    "--out", str(MOBILE / "test/benchmarks/latest-results.candidate.json"),\n    "--supported-only",\n], check=True)\n\nsubprocess.run([\n    "python3", "scripts/compare_benchmark_results.py",\n    "--baseline", str(MOBILE / "test/benchmarks/latest-results.json"),\n    "--candidate", str(MOBILE / "test/benchmarks/latest-results.candidate.json"),\n    "--out", str(MOBILE / "test/benchmarks/latest-results.compare.json"),\n], check=True)

## What To Look For\n- `benchmark-coverage-report.resolved.json`: coverage gaps and unknown labels.\n- `benchmark-dataset-audit.resolved.json`: duplicates, missing URLs, class balance recommendations.\n- `latest-results.candidate.analysis.json`: missed labels and false-positive priorities.\n- `latest-results.compare.json`: baseline vs candidate metric deltas.